In [1]:
import pav

In [2]:
import pandas as pd
import collections
import itertools
import time

import pav

# TODO

For each set of winners
    * for each possible ballot
        * what is the score

take that plus and minus as the possible range of impacts of an incorrect interpretation

for pairs of interpretations then (the reported winners and the runner-up winterset)
    * see how ballots could bridge the gap

For a given winner set, look at the range of possible scores

SPAV one round per winner, vs STV - one round to eliminate one or more lowers

TODO:
* routine to tally election for a given set of winners
* Test identity (`is`) for elements of frozenset from bools_to_set and make bools_to_set actually return and increment counters of the sets, rather than duplicating them
* perhaps sort set by something like sum of plurality totals for each candidate they voted for, and optimize counting with them somehow, bounding how far down the list to go

all winnersets: itertools.combinations(candidatelist, winners)

In [30]:
BCVR = collections.namedtuple('BCVR', ['A', 'B', 'C', 'D'])

In [188]:
import importlib
importlib.reload(pav)

<module 'pav' from '/srv/s/approvalvoting/pr_voting_methods/pav.py'>

In [189]:
pav.tally_pav([BCVR(1, 1, 0, 0)], 2)

1 CVRs, 1 unique selections of candidates

Proportional Approval Voting results

Top 10 scores:
('0.000', ['A', 'B'])
('0.000', ['A', 'C'])
('0.000', ['A', 'D'])
('0.000', ['B', 'C'])
('0.000', ['B', 'D'])
('0.000', ['C', 'D'])

Max score ('0.000', ['A', 'B'])


((frozenset({'A', 'B'}), 0.0),
 [frozenset({'A', 'B'})],
 {frozenset({'A', 'B'}): 0.0,
  frozenset({'A', 'C'}): 0.0,
  frozenset({'A', 'D'}): 0.0,
  frozenset({'B', 'C'}): 0.0,
  frozenset({'B', 'D'}): 0.0,
  frozenset({'C', 'D'}): 0.0})

In [190]:
bs = pav.bools_to_set(BCVR(1, 1, 0, 0))

In [191]:
pav.tally_pav_for_result(set(['A', 'B']), [bs], 2)

1.5

For each set of winners
    * for each possible ballot
        * what is the score

take that plus and minus as the possible range of impacts of an incorrect interpretation

for pairs of interpretations then (the reported winners and the runner-up winterset)
    * see how ballots could bridge the gap


In [95]:
candidatelist = ['A', 'B', 'C', 'D']
num_winners = 2

In [96]:
winnersets = itertools.combinations(candidatelist, num_winners)

In [98]:
print(list(winnersets))

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'C'), ('B', 'D'), ('C', 'D')]


In [159]:
def all_approval_ballots(candidatelist):
    "Yield all possible approval ballots for given list of candidates"

    candidate_or_not = zip([''] * len(candidatelist), candidatelist)
    for decisions in itertools.product(*candidate_or_not):
        yield list(filter(lambda c: c != '', decisions))

In [161]:
list(all_approval_ballots(candidatelist))

[[],
 ['D'],
 ['C'],
 ['C', 'D'],
 ['B'],
 ['B', 'D'],
 ['B', 'C'],
 ['B', 'C', 'D'],
 ['A'],
 ['A', 'D'],
 ['A', 'C'],
 ['A', 'C', 'D'],
 ['A', 'B'],
 ['A', 'B', 'D'],
 ['A', 'B', 'C'],
 ['A', 'B', 'C', 'D']]

In [139]:
cvr = ['A', 'B', 'C', 'D']

In [140]:
pav.tally_pav_for_result(winnerset, [set(cvr)], num_winners)

1.5

In [171]:
def maxdeltascore(candidatelist, winners, altwinners, num_winners):
    winnerset = set(winners)
    altwinnerset = set(altwinners)
    maxdelta = -100
    for cvr in all_approval_ballots(candidatelist):
        cvr_score = pav.tally_pav_for_result(winnerset, [set(cvr)], num_winners)
        cvr_score2 = pav.tally_pav_for_result(altwinnerset, [set(cvr)], num_winners)
        for acvr in all_approval_ballots(candidatelist):
            acvr_score = pav.tally_pav_for_result(winnerset, [set(acvr)], num_winners)
            acvr_score2 = pav.tally_pav_for_result(altwinnerset, [set(acvr)], num_winners)
            # print("%.4f, %s, %.4f, %s" % (cvr_score, cvr, acvr_score, acvr))
            delta = (cvr_score - cvr_score2) - (acvr_score - acvr_score2)
            if acvr != cvr  and  delta > maxdelta:
                maxdelta = delta
                maxpairs = (cvr, acvr)
                print(f'new max delta = {maxdelta:.2f}: {cvr_score:.2f}, {cvr_score2:.2f}, {acvr_score:.2f}, {acvr_score2:.2f}, {cvr}, {acvr}')
            # pav.tally_pav_for_result(set(['A', 'B']), [set(cvr)], len(candidatelist))

    print("max delta: %.4f for winnerset %s vs altwinnerset %s via %s" % (maxdelta, winnerset, altwinnerset, maxpairs))

In [185]:
e1_cvrs = [BCVR(0,1,1,0)] * 10000 + [BCVR(1,1,0,0)] * 1500 + [BCVR(0,0,1,1)] * 800

In [186]:
len(e1_cvrs)

12300

In [175]:
%pdb

Automatic pdb calling has been turned ON


In [187]:
pav.tally_pav(e1_cvrs, 2)

12300 CVRs, 3 unique selections of candidates

Proportional Approval Voting results

Top 10 scores:
('0.000', ['A', 'B'])
('0.000', ['A', 'C'])
('0.000', ['A', 'D'])
('0.000', ['B', 'C'])
('0.000', ['B', 'D'])
('0.000', ['C', 'D'])

Max score ('0.000', ['A', 'B'])


((frozenset({'A', 'B'}), 0.0),
 [frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', 'C'}),
  frozenset({'B', '

In [180]:
b = BCVR(1,1,0,0)

In [160]:
maxdeltascore(candidatelist, ['A', 'B'], ['C', 'D'], 2)

max delta: 3.0000 for winnerset {'A', 'B'} vs altwinnerset {'C', 'D'} via (['A', 'B'], ['C', 'D'])


In [172]:
maxdeltascore(candidatelist, ['A', 'B'], ['C', 'D'], 2)

new max delta = 1.00: 0.00, 0.00, 0.00, 1.00, [], ['D']
new max delta = 1.50: 0.00, 0.00, 0.00, 1.50, [], ['C', 'D']
new max delta = 2.00: 1.00, 0.00, 0.00, 1.00, ['B'], ['D']
new max delta = 2.50: 1.00, 0.00, 0.00, 1.50, ['B'], ['C', 'D']
new max delta = 3.00: 1.50, 0.00, 0.00, 1.50, ['A', 'B'], ['C', 'D']
max delta: 3.0000 for winnerset {'A', 'B'} vs altwinnerset {'C', 'D'} via (['A', 'B'], ['C', 'D'])


In [162]:
maxdeltascore(candidatelist, ['A', 'B'], ['B', 'C'], 2)

max delta: 2.0000 for winnerset {'A', 'B'} vs altwinnerset {'B', 'C'} via (['A'], ['C'])


In [158]:
maxdeltascore(candidatelist, ['A', 'B'], ['B', 'C'], 2)

max delta: 2.0000 for winnerset {'A', 'B'} vs altwinnerset {'B', 'C'} via (['A', 'D'], ['C', 'D'])


## Another approach

In [31]:
binary_cvrs = [BCVR(1, 1, 0, 0)] * 5 + [BCVR(1, 0, 1, 0)] * 17 + [BCVR(0, 0, 0, 1)] * 8

In [48]:
len(binary_cvrs)

30

In [52]:
c = binary_cvrs.copy()

In [53]:
reported_cvr = c.pop()

BCVR(A=0, B=0, C=0, D=1)

In [55]:
reported_cvr = BCVR(A=0, B=0, C=0, D=1)

In [54]:
for cvr in allbcvrs:
    winner, cvrs, scores = pav.tally_pav(c, 2)
    print("cvr: margin(scores)

29

In [33]:
winner, cvrs, scores = pav.tally_pav(binary_cvrs, 2)

30 CVRs, 3 unique selections of candidates

Proportional Approval Voting results

Top 10 scores:
('30.500', ['A', 'C'])
('30.000', ['A', 'D'])
('25.000', ['C', 'D'])
('24.500', ['A', 'B'])
('22.000', ['B', 'C'])
('13.000', ['B', 'D'])

Max score ('30.500', ['A', 'C'])


In [37]:
winner, cvrs, scores = pav.tally_pav([BCVR(1, 1, 0, 0)] * 5 + [BCVR(1, 0, 1, 0)] * 17 + [BCVR(0, 0, 0, 1)] * 7, 2)

29 CVRs, 3 unique selections of candidates

Proportional Approval Voting results

Top 10 scores:
('30.500', ['A', 'C'])
('29.000', ['A', 'D'])
('24.500', ['A', 'B'])
('24.000', ['C', 'D'])
('22.000', ['B', 'C'])
('12.000', ['B', 'D'])

Max score ('30.500', ['A', 'C'])


In [39]:
scores

{frozenset({'A', 'B'}): 24.5,
 frozenset({'A', 'C'}): 30.5,
 frozenset({'A', 'D'}): 29.0,
 frozenset({'B', 'C'}): 22.0,
 frozenset({'B', 'D'}): 12.0,
 frozenset({'C', 'D'}): 24.0}

In [40]:
sorted(scores.items(), key=lambda tuple: tuple[1], reverse=True)[:2]

[(frozenset({'A', 'C'}), 30.5), (frozenset({'A', 'D'}), 29.0)]

In [41]:
def margin(scores):
    top_two = sorted(scores.items(), key=lambda tuple: tuple[1], reverse=True)[:10]
    return top_two[0][1] - top_two[1][1]

In [42]:
margin(scores)

1.5

In [44]:
allbcvrs = list(BCVR(a, b, c, d) for a in (0, 1) for b in (0, 1) for c in (0, 1) for d in (0, 1))

In [45]:
allbcvrs

[BCVR(A=0, B=0, C=0, D=0),
 BCVR(A=0, B=0, C=0, D=1),
 BCVR(A=0, B=0, C=1, D=0),
 BCVR(A=0, B=0, C=1, D=1),
 BCVR(A=0, B=1, C=0, D=0),
 BCVR(A=0, B=1, C=0, D=1),
 BCVR(A=0, B=1, C=1, D=0),
 BCVR(A=0, B=1, C=1, D=1),
 BCVR(A=1, B=0, C=0, D=0),
 BCVR(A=1, B=0, C=0, D=1),
 BCVR(A=1, B=0, C=1, D=0),
 BCVR(A=1, B=0, C=1, D=1),
 BCVR(A=1, B=1, C=0, D=0),
 BCVR(A=1, B=1, C=0, D=1),
 BCVR(A=1, B=1, C=1, D=0),
 BCVR(A=1, B=1, C=1, D=1)]

In [47]:
cvrs

[frozenset({'A', 'B'}),
 frozenset({'A', 'B'}),
 frozenset({'A', 'B'}),
 frozenset({'A', 'B'}),
 frozenset({'A', 'B'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'D'}),
 frozenset({'D'}),
 frozenset({'D'}),
 frozenset({'D'}),
 frozenset({'D'}),
 frozenset({'D'}),
 frozenset({'D'})]

In [38]:
cvrs

[frozenset({'A', 'B'}),
 frozenset({'A', 'B'}),
 frozenset({'A', 'B'}),
 frozenset({'A', 'B'}),
 frozenset({'A', 'B'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'D'}),
 frozenset({'D'}),
 frozenset({'D'}),
 frozenset({'D'}),
 frozenset({'D'}),
 frozenset({'D'}),
 frozenset({'D'})]

In [28]:
bcvrs, df = pav.read_cvrs("Westminster_Adams.csv", 3)

Plurality results in order
Anita_Seitz         4921
Shannon_Bird        4823
Maria_De_Cambra     3921
Mark_Clark          3500
David_DeMott        3126
Debbie_Bergamo      2668
Mike_Melvin         2112
Jason_Blanckaert    2026
Steve_Caulk         2026
Nathan_Pearce       1229
dtype: int64



In [29]:
len(bcvrs)

TypeError: object of type 'map' has no len()

> <ipython-input-29-f119513790e9>(1)<module>()
----> 1 len(bcvrs)

ipdb> q


# %pdb

In [25]:
winner, cvrs, scores = pav.tally_pav(bcvrs, 3)

UnboundLocalError: local variable 'cvr' referenced before assignment

> /srv/s/approvalvoting/pr_voting_methods/pav.py(144)tally_pav()
    142 
    143     for cvr in binary_cvrs:
--> 144         cvrs.append(bools_to_set(cvr))
    145 
    146     all_candidates = type(cvr)._fields

ipdb> cvr
*** NameError: name 'cvr' is not defined
ipdb> binary_cvrs
ipdb> l
    139     """
    140 
    141     cvrs = []
    142 
    143     for cvr in binary_cvrs:
--> 144         cvrs.append(bools_to_set(cvr))
    145 
    146     all_candidates = type(cvr)._fields
    147     uniq = collections.Counter(cvrs)
    148     print("{} CVRs, {} unique selections of candidates".format(len(cvrs), len(uniq)))
    149 

ipdb> l-
*** Error in argument: '-'
ipdb> l -
*** Error in argument: '-'
ipdb> bt
  <ipython-input-25-e6e49e003e4a>(1)<module>()
----> 1 winner, cvrs, scores = pav.tally_pav(bcvrs, 3)

> /srv/s/approvalvoting/pr_voting_methods/pav.py(144)tally_pav()
    142 
    143     for cvr in binary_cvrs:
--> 144         cvrs.append(bools_to_set(cvr))
    145 
    146     al

In [26]:
bcvrs

In [27]:
list(bcvrs)

[]